In [1]:
! conda install -c conda-forge keras==2.3.1 -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ubuntu/miniconda3/envs/py3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipython-7.20.0             |   py37h888b3d9_2         1.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py37h06a~ --> conda-forge::certifi-2020.12.5-py37h89c1867_1
  ipython                             7.12.0-py37h5ca1d4c_0 --> 7.20.0-py37h888b3d9_2

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certif

In [4]:

# for keras 2.3.1
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def toPredictDF(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):
        #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)
            ready_df =df[[ "reps"]]
            df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
            df_new.columns = df_new.columns.astype(str)
            dl = learner.dls.test_dl(df_new)
            _preds,_none ,_y = learner.get_preds(dl=dl, with_decoded=True)
            df.drop(columns=['reps','length'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path)
    print("Complete")

def mergeDF(path,to_dir,file_name):
    appended_data = []
    for infile in glob.glob(path):
        #print(infile)
        data = pd.read_pickle(infile)
        # store DataFrame in list
        appended_data.append(data)
    result_path=to_dir+"/"+file_name
    print("Save:",result_path)
    appended_data = pd.concat(appended_data)
    appended_data.sort_values(by=['ID'], inplace=True)
    appended_data.to_pickle(result_path)
    return appended_data

def predict_CNN(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):  
     #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)
            ready_df =df[[ "reps"]]
            
            X= np.array(df['reps'].to_list())
            X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
            y_probas = learner.predict(X_test)
            threshold = 0.5
            _y = np.where(y_probas > threshold, 1, 0)
            df.drop(columns=['reps'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path) 

    print("Complete")



In [1]:
# load the baseline model
#learn.load('TubularLearner.fastAI._stage1')
deployed_path = "/mnt/vdb/thesis/CustomCNN.Adam.512_1211.hdf5"
#deployed_path = "TubularLearner.fastAI._stage2.pth"
#learner = load_model(deployed_path)


In [5]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    

#import tensorflow as tf

#with tf.device('/cpu:0'):
learner = load_model(deployed_path)

In [6]:
# summarize model.
learner.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1900, 1)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1900, 512)         4608      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 633, 512)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 633, 256)          524544    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 316, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 80896)             0         
_________________________________________________________________
dense (Dense)                (None, 1211)             

## DECockRoach

In [7]:
existing_result = []
for infile in glob.glob("/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [ ]:
# Predict
predict_CNN("/mnt/vdb/DECockroach/cd100/reps/*.pkl","/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam",existing_result)

In [ ]:
# write result
result_df =mergeDF("/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam/*.pkl","/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam","DECockroach.len15.MLResult.plk")
result_df

In [ ]:
result_df[result_df["class"]== 1]

## BAT

In [ ]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Bat/cd100/result_2_CNN_Adam/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

In [ ]:
# Predict
predict_CNN("/mnt/vdb/Bat/cd100/reps/*.pkl","/mnt/vdb/Bat/cd100/result_2_CNN_Adam",existing_result)

In [ ]:
# write result
result_df =mergeDF("/mnt/vdb/Bat/cd100/result_2_CNN_Adam/*.pkl","/mnt/vdb/Bat/cd100/result_2_CNN_Adam","Bat.len10.MLResult.plk")
result_df